# Project Description:

## CSE 250 A: Artificial Intelligence - Probabilistic Reasoning and Decision Making;  


### HW 1 Project:
Imagine a game in which you are asked to guess the word w one letter at a time. The rules of this game
are as follows: 

After each letter (A through Z) that you guess, you’ll be told whether the letter appears in
the word and also where it appears. Given the evidence that you have at any stage in this game, the critical
question is what letter to guess next. Solve this question using Bayesian network.

At any point of time, you are given some letters(>=0) of the word, with their positions. You are also given some letters that are not present in the word. The task is to find out the next best guess, given the aforementioned data.

## Import packages

In [314]:
import pandas as pd
import numpy as np
from __future__ import division

## Read file to take input

In [315]:
data = pd.read_csv('hw1_word_counts_05.txt', sep=" ", header = None)
data.columns = ["word", "count"]
# print data.head(10)
number_of_words = data.shape[0]
print "Number of words in the dictionary: %d" %(number_of_words)

Number of words in the dictionary: 6535


## Sort the dataframe and print most-frequent, least-frequent words

In [316]:
data = data.sort_values("count", ascending=False)
print "15 most frequent words are as follows:"
print "-"*20
print data.head(15).to_string(index = False)
print "-"*20+"\n"
print "14 least frequent words are as follows:"
print "-"*20
print data.tail(14).sort_values(["count","word"]).to_string(index = False)
print "-"*20

15 most frequent words are as follows:
--------------------
  word   count
 THREE  273077
 SEVEN  178842
 EIGHT  165764
 WOULD  159875
 ABOUT  157448
 THEIR  145434
 WHICH  142146
 AFTER  110102
 FIRST  109957
 FIFTY  106869
 OTHER  106052
 FORTY   94951
 YEARS   88900
 THERE   86502
 SIXTY   73086
--------------------

14 least frequent words are as follows:
--------------------
  word  count
 BOSAK      6
 CAIXA      6
 MAPCO      6
 OTTIS      6
 TROUP      6
 CCAIR      7
 CLEFT      7
 FABRI      7
 FOAMY      7
 NIAID      7
 PAXON      7
 SERNA      7
 TOCOR      7
 YALOM      7
--------------------


## Calculate prior probability of each word

In [317]:
prob_of_word = []
total_count = data["count"].sum()
print total_count
for row in data["count"]:
    prob_of_word.append(row/total_count)
data['prob_of_word'] = prob_of_word
print data.head()

7664857
       word   count  prob_of_word
5821  THREE  273077      0.035627
5102  SEVEN  178842      0.023333
1684  EIGHT  165764      0.021626
6403  WOULD  159875      0.020858
18    ABOUT  157448      0.020542


## Make data structures to store constraints
### 1. positive_constraints: To store existing letters and their positions
### 2. negative_constraints: To store exception list (letters that did not work)

In [318]:

positions = range(1,6)
chars = "_"*5
#Map of current state that indicates which character occupies which position. Empty space is '_'
positive_constraints = pd.DataFrame({'Position': positions, 'Value':list(chars)})
# print positive_constraints

sets = [set() for i in range(1,6)]
#Map of current state that indicates which character either did not exist or already used in the word
negative_constraints = pd.DataFrame({'Position': positions, 'Value':sets})
print negative_constraints

   Position Value
0         1    {}
1         2    {}
2         3    {}
3         4    {}
4         5    {}


## Calculate values for questions in the table

   ### 1. correctly guessed = "D -- -- -- I"  |   incorrectly guessed = { }

In [319]:

# Sets complement of positive constraints in negative constraints
def set_negative_constraints(position, value):
    for index, row in negative_constraints.iterrows():
        if row['Position'] != position:
            temp = negative_constraints.get_value(row['Position']-1, 'Value')
            temp.add(value)
            negative_constraints.set_value(row['Position']-1, 'Value', temp)
    
# Setting positive constraints
exception_set = set({'D','I'})
positive_constraints.set_value(0, 'Value', "D")
positive_constraints.set_value(3, 'Value', "I")
exception_set = set({'D','I'})
print positive_constraints

for index, row in positive_constraints.iterrows():
    if row['Value'] != '_' and row['Value']!=' ':
        set_negative_constraints(row['Position'], row['Value'])

# Setting negative constraints
neg_list = ['']
if not neg_list:
    exception_set.update(neg_list)
    for i in neg_list:
        print i, type(i)
        for index, row in negative_constraints.iterrows():
            temp = negative_constraints.get_value(index, 'Value')
            temp.add(i)
            negative_constraints.set_value(index, 'Value', temp)

print negative_constraints

   Position Value
0         1     D
1         2     _
2         3     _
3         4     I
4         5     _
   Position   Value
0         1     {I}
1         2  {I, D}
2         3  {I, D}
3         4     {D}
4         5  {I, D}


In [320]:
def positive_satisfied(word, pos_constraint):
    for index, row in pos_constraint.iterrows():
        if row['Value'] != '_':
            if word[row['Position']-1] != row['Value']:
                return 0
    return 1

def negative_satisfied(word, neg_constraint):
    for index, row in neg_constraint.iterrows():
        if len(row['Value'])>0:
            if word[row['Position']-1] in row['Value']:
                return 0
    return 1


def prob_evidence_given_word(word, pos_constraint, neg_constraint):
    return positive_satisfied(word, pos_constraint) and negative_satisfied(word, neg_constraint)


In [321]:
word_evidence= {}
for index, row in data.iterrows():
    word_evidence[row['word']] = prob_evidence_given_word(row['word'], positive_constraints, negative_constraints)

# print word_evidence

In [322]:
# PSEUDOCODE:
# for each alphabet in alphabet_available:
    # find argmax(alphabet): p(alphabet|evidence) = sum over each word: p(word|evidence) * p(alphabet|word) .............(1)
        # Now, p(alphabet|word) = 1 if word contains alphabet else 0 ..................................(2)
        # p(word|evidence) = [p(evidence|word) * p(word)]/[p(evidence)] ...............................(3)
            # Now p(evidence|word) = 1 if evidence agrees with word else 0 ............................(4)
            # p(evidence) = sum over each word : p(word, evidence) 
            #             = sum over each word : p(word) * p(evidence|word) ...........................(5)
            # Put (4) and (5) in (3)
            # Put (2) and (3) in (1)
# prob_alphabets_given_evidence =  pd.DataFrame({'Alphabet':alphabet_list, 'Probability':np.zeros(26)})
# prob_alphabets_given_evidence = pd.DataFrame({'Alphabet':['E'], 'Probability':[0]})

# for index, row in prob_alphabets_given_evidence.iterrows():
#     waste_words = []
#     total_prob_value = 0
#     for data_index, data_row in data.iterrows():
#         word = data_row['word']
#         print word
#         if row['Alphabet'] not in word:
#             # return 0 as per step 2
#             # update alphabets dataframe
#             waste_words.append(word)
#             prob_alphabets_given_evidence.set_value(index, 'Probability',0)
#             continue
            
#         # calculating p(word|evidence)
#         numerator = prob_evidence_given_word(word, positive_constraints, negative_constraints)*data_row['prob_of_word']
#         if numerator == 0:
#             # return 0 as per step 4
#             # update alphabets dataframe
#             waste_words.append(word)
#             prob_alphabets_given_evidence.set_value(index, 'Probability',0)
#             continue
        
#         denominator = 0
#         final_prob_value = 0
#         for data_two_index, data_two_row in data.iterrows():
#             if data_two_row['word'] not in waste_words:
#                 denominator += prob_evidence_given_word(data_two_row['word'], 
#                                                     positive_constraints, 
#                                                     negative_constraints) *data_two_row['prob_of_word']
#         final_prob_value = numerator/denominator
#         total_prob_value+= final_prob_value
#         #update alphabets dataframe
#     print "Probability for: ", row['Alphabet']," is :", total_prob_value
#     prob_alphabets_given_evidence.set_value(index, 'Probability',total_prob_value)

In [311]:
# FILTERING WORD WITH EVIDENCE BEFOREHAND
# PSEUDOCODE:
# for each alphabet in alphabet_available:
    # find argmax(alphabet): p(alphabet|evidence) = sum over each word: p(word|evidence) * p(alphabet|word) .............(1)
        # Now, p(alphabet|word) = 1 if word contains alphabet else 0 ..................................(2)
        # p(word|evidence) = [p(evidence|word) * p(word)]/[p(evidence)] ...............................(3)
            # Now p(evidence|word) = 1 if evidence agrees with word else 0 ............................(4)
            # p(evidence) = sum over each word : p(word, evidence) 
            #             = sum over each word : p(word) * p(evidence|word) ...........................(5)
            # Put (4) and (5) in (3)
            # Put (2) and (3) in (1)
prob_alphabets_given_evidence =  pd.DataFrame({'Alphabet':alphabet_list, 'Probability':np.zeros(26)})

for index, row in prob_alphabets_given_evidence.iterrows():
    if row['Alphabet'] in exception_set: # if they are in one of the constraints, then don't evaluate them
        continue
    total_prob_value = 0
    for data_index, data_row in data.iterrows():
        word = data_row['word']
        if word_evidence[word]==0: # if word does not agree with the evidence, don't evaluate (saves some time)
            continue
        if row['Alphabet'] not in word:
            # return 0 as per step 2
            # update alphabets dataframe
            prob_alphabets_given_evidence.set_value(index, 'Probability',0)
            continue
            
        # calculating p(word|evidence)
        numerator = prob_evidence_given_word(word, positive_constraints, negative_constraints)*data_row['prob_of_word']
        if numerator == 0:
            # return 0 as per step 4
            # update alphabets dataframe
            prob_alphabets_given_evidence.set_value(index, 'Probability',0)
            continue
        
        # if none of the above values are 0, apply Bayes rule
        denominator = 0
        final_prob_value = 0
        for data_two_index, data_two_row in data.iterrows():
            denominator += prob_evidence_given_word(data_two_row['word'], 
                                                    positive_constraints, 
                                                    negative_constraints) *data_two_row['prob_of_word']
        final_prob_value = numerator/denominator
        total_prob_value+= final_prob_value
        #update alphabets dataframe
#     print "Probability for: ", row['Alphabet']," is :", total_prob_value
    prob_alphabets_given_evidence.set_value(index, 'Probability',total_prob_value)

Probability for:  A  is : 0.82068452381
Probability for:  B  is : 0.0171130952381
Probability for:  C  is : 0
Probability for:  E  is : 0.140811011905
Probability for:  F  is : 0
Probability for:  G  is : 0
Probability for:  H  is : 0
Probability for:  J  is : 0.00688244047619
Probability for:  K  is : 0.00316220238095
Probability for:  L  is : 0.0738467261905
Probability for:  M  is : 0.0178571428571
Probability for:  N  is : 0.186011904762
Probability for:  O  is : 0.0453869047619
Probability for:  P  is : 0.00688244047619
Probability for:  Q  is : 0
Probability for:  R  is : 0.180989583333
Probability for:  S  is : 0.739583333333
Probability for:  T  is : 0.0152529761905
Probability for:  U  is : 0.00186011904762
Probability for:  V  is : 0.743675595238
Probability for:  W  is : 0
Probability for:  X  is : 0
Probability for:  Y  is : 0
Probability for:  Z  is : 0


In [325]:
result = prob_alphabets_given_evidence.sort_values("Probability", ascending=False)
print result[['Alphabet', 'Probability']].head(1)

  Alphabet  Probability
0        A     0.820685
